In [7]:
%load_ext autoreload

%autoreload 2

from __future__ import division, print_function

import cPickle as pickle
from glob import glob
import subprocess
import sys
import string
import os
import random
import numpy as np
from os.path import join

import threebody

import logging
logger = logging.getLogger()
#logger.setLevel(logging.DEBUG)
logger.setLevel(logging.INFO)
logging.debug("test")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [45]:
name = "dmtesting"

In [46]:
d = join("derivative-runs",name)
params = pickle.load(open(join(d,"params.pickle"),"rb"))
F = threebody.Fitter(**params)
try:
    F.best_parameters = pickle.load(
        open(join(d,"best_parameters.pickle"),"rb"))
except IOError:
    pass
derivatives = {}
for p in F.parameters:
    derivatives[p] = np.load(join(d,"%s.npy" % p))
print("Goodness of fit:",F.goodness_of_fit())
M, linear_parameters = F.compute_linear_matrix()
lv, linear_parameters = F.compute_linear_parts()
for i, p in enumerate(linear_parameters):
    derivatives[p] = M[:,i]

ps = F.parameters + linear_parameters
Abig = np.array([derivatives[p] 
                 for p in ps], 
                dtype=np.longdouble).T
#print(Abig.shape)
result = threebody.lstsq_with_errors(Abig, 
                                     F.residuals(), 
                                     F.phase_uncerts)

for i,p in enumerate(ps):
    if p in F.parameters:
        v = F.best_parameters[p]
    elif p in linear_parameters:
        v = lv[linear_parameters.index(p)]
    else:
        v = 0
    print(p, 
          threebody.fmt(v-result.x[i], result.uncerts[i]), 
          result.uncerts[i])

Goodness of fit: 39907.0546317
asini_i 1.2175243(2)e0 2.33678894276e-07
pb_i 1.6293909(6)e0 6.02189987355e-07
eps1_i 6.8900(18)e-4 1.82207016017e-07
eps2_i -9.1463(17)e-5 1.73734612523e-08
tasc_i 4.0771638(6)e-1 5.79712914925e-08
acosi_i 1.48954(18)e0 0.000177412511269
q_i 1.37431(4)e-1 3.98411471275e-06
asini_o 7.4672590(11)e1 1.14195575329e-05
pb_o 3.2725652(10)e2 9.95086802009e-05
eps1_o 3.518581(4)e-2 4.36114099251e-08
eps2_o -3.46359(15)e-3 1.45098404179e-07
tasc_o 3.1393480(10)e2 9.94679139442e-05
acosi_o 9.1369(11)e1 0.0114499700449
delta_lan -5.3(7)e-6 6.94278216971e-07
delta -1.67(17)e-6 1.69715666162e-07
dgamma 6(3)e-2 0.0252414357206
dbeta 1.5(5)e-1 0.0482485184572
FD1 4(10)e-1 0.956470105268
FD2 -4(9)e-1 0.928354361472
FD3 2(5)e-1 0.536720263027
FD4 0(15)e-2 0.153722453772
d_DECJ 5.79(17)e-5 1.65403272011e-06
d_DM 0(12)e1 115.878015273
d_PMDEC -1.55(9)e-8 9.28313220634e-10
d_PMRA -9.5(3)e-9 2.52027964627e-10
d_PX -9(6)e0 5.50121269708
d_RAJ -1.71(4)e-5 3.85130386477e-07
f0 

In [47]:
# FIXME
bp = F.best_parameters.copy()
for p in F.parameters:
    i = ps.index(p)
    bp[p] -= result.x[i]
pre = F.goodness_of_fit(F.best_parameters)
print("pre-improvement:", pre)
post = F.goodness_of_fit(bp)
print("post-improvement:", post)
if post<pre:
    with open("derivative-best-parameters.pickle","wb") as f:
        pickle.dump(bp, f)

pre-improvement: 39907.0546317
post-improvement: 7634274.73486


In [48]:
F.parameters

['asini_i',
 'pb_i',
 'eps1_i',
 'eps2_i',
 'tasc_i',
 'acosi_i',
 'q_i',
 'asini_o',
 'pb_o',
 'eps1_o',
 'eps2_o',
 'tasc_o',
 'acosi_o',
 'delta_lan',
 'delta',
 'dgamma',
 'dbeta']